### Disaster Tweets
#### `Hugo Hersemeule, Tom Kubasik`
#### Enoncé Kaggle: https://www.kaggle.com/c/nlp-getting-started/overview

In [1]:
import pandas
import numpy as np

train_data = pandas.read_csv("./data/train.csv")
test_data = pandas.read_csv("./data/test.csv")

L'une des principale difficulté dans le NLP est qu'un algorithme de machine learning ne peut pas traiter des mots, il ne peut traiter que des données numériques. 

On peut utiliser CountVectorizer() pour transformer nos tweets en données numérique.
Il convertit une serie de text en une matrix de comptage de mots.

On utilise 'fit_transform' sur les data d'entrainement, et 'transform' sur les données de test afin de pouvoir tester les données de test avec la même base de mot que les données d'entrainement.

In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

count_vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = count_vectorizer.fit_transform(train_data["text"])
test_vectors = count_vectorizer.transform(test_data["text"])

Avec ces données on fait une régression Ridge.
On peut voir que l'on optient un score en moyenne entre 50 et 60% de bonne reponses

In [3]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, train_vectors, train_data["target"], cv=5, scoring="f1")
print(scores)

clf.fit(train_vectors, train_data["target"])

[0.6025641  0.50126582 0.56985004 0.50781969 0.67275495]


RidgeClassifier()

Une des ammélioration de countVectorizer est d'utiliser la technique de Stemming, cela consiste a enlever les suffix des mots, ainsi "troubling", "troubled", et "trouble" diviennent "troubl" 

In [4]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tomkubasik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()

In [6]:
print(porter.stem("troubled"))

troubl


In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [8]:
x=stemSentence(train_data["text"][0])

In [9]:
print(train_data["text"][0])
print(x)

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
our deed are the reason of thi # earthquak may allah forgiv us all 


In [10]:
stemed_train_data = map(stemSentence, train_data["text"])

On applique le stemming a toutes les phrases et on refait le countVertorizer(), on peut voir une petit augmentation de la fiabilité.

In [11]:
count_vectorizer_stem = feature_extraction.text.CountVectorizer()

train_vectors_stem = count_vectorizer_stem.fit_transform(stemed_train_data)

clf_stem = linear_model.RidgeClassifier()
scores_stem = model_selection.cross_val_score(clf_stem, train_vectors_stem, train_data["target"], cv=5, scoring="f1")
print(scores_stem)

[0.59170507 0.52796053 0.58447489 0.57529611 0.68711656]


Une autre chose possible est d'utiliser la technique de Lematization, cela consiste a regrouper les mots par champs lexicals, ici, on le fait sur les verbe, ainsi "Were", "was", "is" devient "Be".

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('omw-1.4')

punctuations="?:!.,;#"

def lemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    for word in token_words:
        if word in punctuations:
            token_words.remove(word)
    
    lem_sentence=[]
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos='v'))
        lem_sentence.append(" ")
    return "".join(lem_sentence)

lem=lemmaSentence(train_data["text"][0])
print(train_data["text"][0])
print(lem)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tomkubasik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/tomkubasik/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Our Deeds be the Reason of this earthquake May ALLAH Forgive us all 


In [13]:
lem_train_data = map(lemmaSentence, train_data["text"])

count_vectorizer_lem = feature_extraction.text.CountVectorizer()

train_vectors_lem = count_vectorizer_lem.fit_transform(lem_train_data)

clf_lem = linear_model.RidgeClassifier()
scores_lem = model_selection.cross_val_score(clf_lem, train_vectors_lem, train_data["target"], cv=5, scoring="f1")
print(scores_stem)

[0.59170507 0.52796053 0.58447489 0.57529611 0.68711656]


Dans les données on nous donne une localisation sur certain de tweet, une chose interessante serait de regarder si le fait de mettre une localisation ou non est interessant dans la recherche de "Disaster tweets".

Si on a une localisation, on met un YES, si on n'a pas de localisation, on met un NO.

In [14]:
def replaceTextWith1(sentence):
    if sentence == 0:
        return "NO"
    else:
        return "YES"

train_data["location"] = train_data["location"].fillna(0)
train_data["location"] = list(map(replaceTextWith1, train_data["location"]))

# Verif printing
print(train_data["location"])
print(train_data["location"][50])

0       NO
1       NO
2       NO
3       NO
4       NO
        ..
7608    NO
7609    NO
7610    NO
7611    NO
7612    NO
Name: location, Length: 7613, dtype: object
YES


Pour finir, on tente d'entrainer le modele sur deux parametre, la presence ou non de localisation et le texte lematizé. Pour cela on combinent les deux features et on fait une regression.

In [15]:
train_data["text"] = list(map(lemmaSentence, train_data["text"]))
print(train_data["text"])

0       Our Deeds be the Reason of this earthquake May...
1                  Forest fire near La Ronge Sask Canada 
2       All residents ask to 'shelter in place ' be be...
3       13,000 people receive wildfires evacuation ord...
4       Just get send this photo from Ruby Alaska as s...
                              ...                        
7608    Two giant crane hold a bridge collapse into ne...
7609    @ aria_ahrary @ TheTawniest The out of control...
7610    M1.94 [ 01:04 UTC ] 5km S of Volcano Hawaii ht...
7611    Police investigate after an e-bike collide wit...
7612    The Latest More Homes Razed by Northern Califo...
Name: text, Length: 7613, dtype: object


In [16]:
import pandas as pd
from sklearn.pipeline import FeatureUnion
from scipy.sparse import csr_matrix, hstack
 

# traitement du text
vectorizer = feature_extraction.text.CountVectorizer()
train_vector_final = vectorizer.fit_transform(train_data["text"])

vectorizer_location = feature_extraction.text.CountVectorizer()
train_vector_location = vectorizer_location.fit_transform(train_data["location"])

combined_features= hstack([train_vector_final, train_vector_location], 'csr')


In [17]:
clf_final = linear_model.RidgeClassifier()
scores_final = model_selection.cross_val_score(clf_final, combined_features, train_data["target"], cv=5, scoring="f1")
print(scores_final)

[0.59645853 0.53078203 0.5797546  0.56091148 0.68735806]


On peut voir que qu'il y a peut d'ammélioration... on peut en deduire que la presence de localisation n'influe pas ou peut sur la nature du Tweet, peut etre qu'il faudrait prendre les localisations en elle même plutot qu'un boolean "Localisation indiquée ou non". Pour continuer on pourrait regarder la presence ou non de majuscule dans les tweets, les #Hashtag qui sont propres a la plateforme, Il reste une donnée non exploitée qui est la collomne "keyword".